In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from random import random
# For plotting Gaussian contours
from plotting_util import plot_gaussian_from_points
from plotting_util import plot_gaussian_from_parameters
import pandas as pd

from numpy.linalg import inv
import scipy.stats as stats

import time 



In [2]:
def sampling_s_ty(sigma_O, mu_s, sigma_s, t):
    ### sampling variable s=s1,s2 from condtion t and y(eliminated in this case)
    ### Input: sigma_O, mu_s, sigma_s, t
    ### Ontput: one vrs with Gaussian(mu_s_ty, sigma_s_ty)
    A = np.array([[1,-1]])
    sigma_s_ty = inv(inv(sigma_s) + A.T * (1/sigma_O) @ A)
    mu_s_ty = sigma_s_ty @ (inv(sigma_s) @ mu_s + A.T * (1/sigma_O) * t)
    
    return stats.multivariate_normal.rvs(mu_s_ty.reshape(-1), sigma_s_ty)

In [3]:
def sampling_t_sy(y, s_1, s_2, sigma_O):
    ### sampling variable t from condtion =s1,s2 and y
    ### Input: y, s_1, s_2, sigma_O
    ### Ontput: one vrs with truncated Gaussian(a, b, loc, scale)
    loc = s_1 - s_2
    scale = sigma_O
    if y == 1:
        myclip_a = 0
        myclip_b = np.Inf
    elif y== -1:
        myclip_a = -np.Inf
        myclip_b = 0
    
    a, b = (myclip_a - loc) / scale, (myclip_b - loc) / scale
    return stats.truncnorm.rvs(a, b, loc, scale)


In [4]:
def GibbsSampling(initial_point, y, mu_s, sigma_s, sigma_O, num_samples):
    ### Input: initial points for s_1, s_2, y, mu_s, sigma_s, sigma_O, num_samples
    ### Output: samples for s_1 and s_2
    samples_s_1 = []
    samples_s_2 = []
    s_1 = initial_point[0]
    s_2 = initial_point[1]
    for i in range(num_samples):
        t = sampling_t_sy(y, s_1, s_2, sigma_O)
        s_1,s_2 = sampling_s_ty(sigma_O, mu_s, sigma_s, t)
        samples_s_1.append(s_1)
        samples_s_2.append(s_2)
    return samples_s_1,samples_s_2

In [5]:
s0 = [0,0]
num_samples = 5000
y = 1
mu_s = np.array([[1, 1]]).T
sigma_s = np.array([[1,0],[0,1]])
sigma_O = np.array([1])

t0 = time.time()
vec_s1, vec_s2 = GibbsSampling(s0, y, mu_s, sigma_s, sigma_O, num_samples)
t = int((time.time() - t0) * 1000)


In [6]:
plt.grid()
plt.plot(vec_s1)
plt.plot(vec_s2)
plt.xlabel("Number of samples")
plt.ylabel("s")
plt.title("Samples of s")
print(np.mean(vec_s1))
print(np.mean(vec_s2))
plt.legend(['s1', 's2'], loc='upper right')

1.4382327460014743
0.5410951327746915


In [19]:
burn = 1000
burn_vec_s1 = vec_s1[burn:]
burn_vec_s2 = vec_s2[burn:]

plt.grid()
plt.plot(burn_vec_s1)
plt.plot(burn_vec_s2)
plt.xlabel("Number of samples after burn in")
plt.ylabel("s")
plt.title("Samples of s")
print(np.mean(burn_vec_s1))
print(np.mean(burn_vec_s2))
plt.legend(['s1', 's2'], loc='upper right')

1.4518065190298794
0.5419278284604404


In [28]:
from scipy.stats import truncnorm, norm
def MultiplyGauss(m1,s1,m2,s2):
    ### Input: mean and covariance of two Gaussian distributions
    ### Output: mean and covariance of the product of two Gaussian distributions
    sigma = s1*s2/(s1+s2)
    mu = m1*s2/(s1+s2) + m2*s1/(s1+s2)
    return mu, sigma
def DivideGauss(m1,s1,m2,s2):
    ### Input: mean and covariance of two Gaussian distributions
    ### Output: mean and covariance of the division of two Gaussian distributions
    mu,sigma = MultiplyGauss(m1,s1,m2,-s2)
    return mu, sigma
def truncGauss(a, b, m0, s0):
    a_scaled , b_scaled = (a - m0) / np.sqrt(s0), (b - m0) / np.sqrt(s0)
    m = truncnorm.mean(a_scaled , b_scaled , loc=m0, scale=np.sqrt(s0))
    s = truncnorm.var(a_scaled , b_scaled , loc=m0, scale=np.sqrt(s0))
    return m, s
m_s1 = 1
s_s1 = 1
m_s2 = 1
s_s2 = 1
s_ts = 1
y = 1

In [29]:
def MessagePassing(m1,v1,m2,v2,y0):
    msg3_m = m1 # mean of message
    msg3_s = v1 # variance of message
    msg4_m = m2 # mean of message
    msg4_s = v2 # variance of message
    
    # Message msg5 and msg6
    msg5_m = msg3_m # mean of message
    msg5_s = msg3_s # variance of message
    msg6_m = msg4_m # mean of message
    msg6_s = msg4_s # variance of message

    # Message msg7
    msg7_m = m1 - m2
    msg7_s = s_ts + v1 + v2

    # p_t
    a, b = 0, np.Inf
    p_t_m, p_t_s = truncGauss(a, b, msg7_m, msg7_s)
    
    # Message msg8
    msg8_m, msg8_s = DivideGauss(p_t_m, p_t_s, msg7_m, msg7_s)
    
    # Message msg9 and msg10
    msg9_m = m2+msg8_m
    msg9_s = s_ts+v2+msg8_s
    msg10_m = m1-msg8_m
    msg10_s = s_ts+v1+msg8_s

    # p_s1 and p_s2
    p_s1_m, p_s1_s = MultiplyGauss(m1, v1, msg9_m, msg9_s)
    p_s2_m, p_s2_s = MultiplyGauss(m2, v2, msg10_m, msg10_s)

    return p_s1_m, p_s1_s, p_s2_m, p_s2_s
pm_s1, ps_s1, pm_s2, ps_s2 = MessagePassing(m_s1, s_s1, m_s2, s_s2, s_ts)
print(pm_s1, ps_s1, pm_s2, ps_s2)

1.4606588659617807 0.7877934092108061 0.5393411340382191 0.7877934092108061


In [30]:
def cvrt_gaussian(s, mean, var, num_samples, burn):
    
    x = np.linspace(mean - var * 3, mean + var * 3, num_samples - burn)
    y = stats.norm.pdf(x, loc=mean, scale=var)
    plt.plot(x, y)
    plt.hist(s, bins=20, density=True, color='lightblue')
    plt.grid()
    plt.title("number_samples=" + str(num_samples) + "," + "(" + str(t) + " ms)")
    

In [31]:
cvrt_gaussian(burn_vec_s1, np.mean(burn_vec_s1), np.std(burn_vec_s1), num_samples, burn)

In [34]:
# Calculate the parameters of the approximated pdfs
s1_mean = np.mean(burn_vec_s1)
s1_std = np.std(burn_vec_s1)
s2_mean = np.mean(burn_vec_s2)
s2_std = np.std(burn_vec_s2)
    
# Plot the pdfs
x = np.linspace(1 - 1 * 3, 1 + 1 * 3, num_samples - burn)
plt.plot(x, stats.norm.pdf(x, loc=1, scale=1), label="prior")
plt.plot(x, stats.norm.pdf(x, loc=s1_mean, scale=s1_std), label="s1 posterior")
plt.plot(x, stats.norm.pdf(x, loc=s2_mean, scale=s2_std), label="s2 posterior")
plt.plot(x, stats.norm.pdf(x, loc=pm_s1, scale= np.sqrt(ps_s1)), label="massage passing s1",linestyle='--')
plt.plot(x, stats.norm.pdf(x, loc=pm_s2, scale= np.sqrt(ps_s2)), label="massage passing s2",linestyle='--')
plt.grid()
plt.legend()
plt.show()


# SerieA Task

In [83]:
import pandas as pd
df = pd.read_csv ('SerieA.csv')
#df = pd.read_csv ('england-premier-league-2020-2021.csv')
df

yyyy-mm-dd  HH:MM       team1     team2  score1  score2
0    2018-08-18  18:00      Chievo  Juventus       2       3
1    2018-08-18  20:30       Lazio    Napoli       1       2
2    2018-08-19  18:00      Torino      Roma       0       1
3    2018-08-19  20:30    Sassuolo     Inter       1       0
4    2018-08-19  20:30       Parma   Udinese       2       2
..          ...    ...         ...       ...     ...     ...
375  2019-05-26  20:30        Roma     Parma       2       1
376  2019-05-26  20:30       Inter    Empoli       2       1
377  2019-05-26  20:30  Fiorentina     Genoa       0       0
378  2019-05-26  20:30    Cagliari   Udinese       1       2
379  2019-05-26  20:30    Atalanta  Sassuolo       3       1

[380 rows x 6 columns]

In [84]:
cols = ['team1', 'team2', 'score1', 'score2']

#cols = ['HOME_TEAM', 'AWAY_TEAM', 'FTHG' ,'FTAG']
df = df[cols]
df.rename(columns = {'HOME_TEAM':'team1', 'AWAY_TEAM':'team2', 'FTHG':'score1', 'FTAG': 'score2'}, inplace = True)
df.head()

C:\Users\Weilong Chen\AppData\Local\Temp\ipykernel_6864\3983290589.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'HOME_TEAM':'team1', 'AWAY_TEAM':'team2', 'FTHG':'score1', 'FTAG': 'score2'}, inplace = True)


team1     team2  score1  score2
0    Chievo  Juventus       2       3
1     Lazio    Napoli       1       2
2    Torino      Roma       0       1
3  Sassuolo     Inter       1       0
4     Parma   Udinese       2       2

In [85]:
team1 = set(df['team1'])
team2 = set(df['team2'])
teams = team1.union(team2)

# convert team's name into indices
name_dictionary = dict(zip(teams, range(len(teams))))
print(name_dictionary)

# create skill list [[mean_0,var_0], [mean_1, var_1]...]
skill_list = np.ones((len(teams), 2))


{'Genoa': 0, 'Chievo': 1, 'Bologna': 2, 'Atalanta': 3, 'Inter': 4, 'Empoli': 5, 'Udinese': 6, 'Lazio': 7, 'Sampdoria': 8, 'Torino': 9, 'Spal': 10, 'Cagliari': 11, 'Milan': 12, 'Roma': 13, 'Sassuolo': 14, 'Juventus': 15, 'Fiorentina': 16, 'Frosinone': 17, 'Napoli': 18, 'Parma': 19}


In [86]:
# shuffle

# df = df.sample(frac=1, replace=True, random_state=1).reset_index(drop=True)
# df = df[cols]
# df.head()

In [87]:
s0 = [0,0]
num_samples = 750
sigma_O = np.array([1])
burn = 20



for i in range(len(df)):
    team1 = df.iloc[i,0] # name in str
    team2 = df.iloc[i,1]
    score1 = df.iloc[i,2] # score in int
    score2 = df.iloc[i,3]
    prior1 = skill_list[name_dictionary[team1]]
    prior2 = skill_list[name_dictionary[team2]]
    mu_s = np.array([[prior1[0], prior2[0]]]).T
    sigma_s = np.array([[prior1[1],0],[0,prior2[1]]])
    
    if score1 > score2:
        y = 1
        vec_s1, vec_s2 = GibbsSampling(s0, y, mu_s, sigma_s, sigma_O, num_samples)
        vec_s1 = vec_s1[burn:]
        vec_s2 = vec_s2[burn:]
        posterior1 = [np.mean(vec_s1), np.var(vec_s1)]
        posterior2 = [np.mean(vec_s2), np.var(vec_s2)]
    elif score1 < score2:
        y = -1
        vec_s1, vec_s2 = GibbsSampling(s0, y, mu_s, sigma_s, sigma_O, num_samples)
        vec_s1 = vec_s1[burn:]
        vec_s2 = vec_s2[burn:]
        posterior1 = [np.mean(vec_s1), np.var(vec_s1)]
        posterior2 = [np.mean(vec_s2), np.var(vec_s2)]
    else:
        posterior1 = prior1
        posterior2 = prior2
    
    skill_list[name_dictionary[team1]] = posterior1
    skill_list[name_dictionary[team2]] = posterior2
    
    

In [88]:
skill_team = dict(zip(teams, skill_list[:,0]))

# saving ranking 20/21
skill_team20_21 = skill_team
skill_team20_21

# sort dictionary by values in reverse order
ranking20_21 = dict(sorted(skill_team20_21.items(), key=lambda item: item[1], reverse=True))

ranking20_21 = pd.DataFrame.from_dict(ranking20_21, orient='index')
ranking20_21

0
Juventus    2.077602
Napoli      1.850073
Milan       1.706648
Atalanta    1.672545
Inter       1.617305
Roma        1.545040
Torino      1.503782
Lazio       1.205060
Sampdoria   0.980042
Bologna     0.960374
Spal        0.748923
Empoli      0.748745
Parma       0.683631
Sassuolo    0.662174
Genoa       0.652995
Udinese     0.621636
Fiorentina  0.603156
Cagliari    0.597574
Frosinone   0.005427
Chievo     -0.333065

In [180]:
# do the predictions

df = pd.read_csv ('SerieA.csv')
cols = ['team1', 'team2', 'score1', 'score2']
#cols = ['HOME_TEAM', 'AWAY_TEAM', 'FTHG' ,'FTAG']
df = df[cols]
df.rename(columns = {'HOME_TEAM':'team1', 'AWAY_TEAM':'team2', 'FTHG':'score1', 'FTAG': 'score2'}, inplace = True)
df.head()



team1     team2  score1  score2
0    Chievo  Juventus       2       3
1     Lazio    Napoli       1       2
2    Torino      Roma       0       1
3  Sassuolo     Inter       1       0
4     Parma   Udinese       2       2

In [181]:
skill_team

{'Genoa': 0.5841826468789159,
 'Chievo': -0.20395635323567754,
 'Bologna': 0.5340750278895166,
 'Atalanta': 1.370793665912648,
 'Inter': 1.4380908824699754,
 'Empoli': 0.47178053528333863,
 'Udinese': 0.5054843229998963,
 'Lazio': 1.0453143875319817,
 'Sampdoria': 0.8630508254737864,
 'Torino': 1.189198232916116,
 'Spal': 0.5987043011787087,
 'Cagliari': 0.6703655412556433,
 'Milan': 1.2310472053738273,
 'Roma': 1.100975198241981,
 'Sassuolo': 0.493364522135871,
 'Juventus': 1.75787798376653,
 'Fiorentina': 0.46241612036367935,
 'Frosinone': 0.15500990428651643,
 'Napoli': 1.6072028362838042,
 'Parma': 0.5562797099677215}

In [182]:
# inherinting
# skill_team['Norwich City'] = skill_team.pop('Fulham')
# skill_team['Watford'] = skill_team.pop('Sheffield United')
# skill_team['Brentford'] = skill_team.pop('West Bromwich Albion')

# update name)dict
team1 = set(df['team1'])
team2 = set(df['team2'])
teams = team1.union(team2)
print(teams)

# convert team's name into indices
name_dictionary = dict(zip(teams, range(len(teams))))

{'Genoa', 'Chievo', 'Bologna', 'Atalanta', 'Inter', 'Empoli', 'Udinese', 'Lazio', 'Sampdoria', 'Torino', 'Spal', 'Cagliari', 'Milan', 'Roma', 'Sassuolo', 'Juventus', 'Fiorentina', 'Frosinone', 'Napoli', 'Parma'}


In [183]:
def predictor(mu_s, sigma_s, draw_threshold):
    # mu_s in shape (1,2), sigma_s in shape (2,2)
    s1_pred = np.random.normal(mu_s[0], np.sqrt(sigma_s[0][0]), 50)
    s2_pred = np.random.normal(mu_s[1], np.sqrt(sigma_s[1][1]), 50)
    t_pred =  np.mean(s1_pred-s2_pred)
    if draw_threshold > abs(t_pred):
        t_pred = 0
    
    y_pred = np.sign(t_pred)
    return y_pred

In [184]:
### In this cell, the only difference is to insert one prediction step.

# initialize priors
# skill_list = np.ones((len(teams), 2))

# hyperparameters
s0 = [0,0]
num_samples = 750
sigma_O = np.array([1])
burn = 20
draw_threshold = 0.04

# create pred_
predictions = [] 


for i in range(len(df)):
    team1 = df.iloc[i,0] # name in str
    team2 = df.iloc[i,1]
    score1 = df.iloc[i,2] # score in int
    score2 = df.iloc[i,3]
    prior1 = skill_list[name_dictionary[team1]]
    prior2 = skill_list[name_dictionary[team2]]
    if score1-score2 == 2:
        mu_s = np.array([[1.1*prior1[0], prior2[0]/1.1]]).T
    if score1-score2 > 2:
        mu_s = np.array([[1.2*prior1[0], prior2[0]/1.2]]).T
    if score2-score1 == 2:
        mu_s = np.array([[prior1[0]/1.1, prior2[0]*1.1]]).T
    if score2-score1 > 2:
        mu_s = np.array([[prior1[0]/1.2, prior2[0]*1.2]]).T
    else:
        mu_s = np.array([[prior1[0], prior2[0]]]).T
    sigma_s = np.array([[prior1[1],0],[0,prior2[1]]])

    
    predictions.append(predictor(mu_s, sigma_s, draw_threshold))
    
    if score1 > score2:
        y = 1
        vec_s1, vec_s2 = GibbsSampling(s0, y, mu_s, sigma_s, sigma_O, num_samples)
        vec_s1 = vec_s1[burn:]
        vec_s2 = vec_s2[burn:]
        posterior1 = [np.mean(vec_s1), np.var(vec_s1)]
        posterior2 = [np.mean(vec_s2), np.var(vec_s2)]
    elif score1 < score2:
        y = -1
        vec_s1, vec_s2 = GibbsSampling(s0, y, mu_s, sigma_s, sigma_O, num_samples)
        vec_s1 = vec_s1[burn:]
        vec_s2 = vec_s2[burn:]
        posterior1 = [np.mean(vec_s1), np.var(vec_s1)]
        posterior2 = [np.mean(vec_s2), np.var(vec_s2)]
    else:
        posterior1 = prior1
        posterior2 = prior2
    
    skill_list[name_dictionary[team1]] = posterior1
    skill_list[name_dictionary[team2]] = posterior2

In [185]:
# true values
y_true = []
for i in range(len(df)):
    
    score1 = df.iloc[i,2] # score in int
    score2 = df.iloc[i,3]
    
    if score1 > score2:
        y_true.append(1)
        
    elif score1 < score2:
        y_true.append(-1)
    else:
        y_true.append(0)
        
y_true

[-1,
 -1,
 -1,
 1,
 0,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 0,
 -1,
 -1,
 1,
 -1,
 0,
 0,
 1,
 1,
 -1,
 0,
 1,
 0,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 0,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 0,
 0,
 -1,
 0,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 0,
 -1,
 0,
 -1,
 -1,
 0,
 0,
 1,
 0,
 1,
 -1,
 0,
 0,
 -1,
 0,
 1,
 1,
 0,
 -1,
 1,
 1,
 1,
 0,
 1,
 1,
 -1,
 0,
 -1,
 -1,
 -1,
 0,
 -1,
 0,
 -1,
 1,
 1,
 1,
 0,
 0,
 -1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 -1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 -1,
 1,
 1,
 0,
 0,
 0,
 -1,
 0,
 1,
 0,
 0,
 1,
 -1,
 0,
 1,
 -1,
 1,
 -1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 -1,
 1,
 -1,
 0,
 0,
 1,
 0,
 1,
 -1,
 1,
 -1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 -1,
 -1,
 0,
 0,
 -1,
 1,
 1,
 1,
 1,
 -1,
 0,
 -1,
 0,
 0,
 0,
 1,
 -1,
 1,
 1,
 1,
 0,
 -1,
 -1,
 -1,
 0,
 1,
 -1,
 -1,
 0,
 1,
 0,
 0,
 0,
 0,
 -1,
 0,
 -1,
 -1,
 1,
 -1,
 0,
 -1,
 0,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 

In [186]:
predictions

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 0,
 -1.0,
 1.0,
 -1.0,
 0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 0,
 -1.0,


In [187]:
array1 = np.array(predictions)
array2 = np.array(y_true)

acc_array = np.subtract(array1, array2)

acc = 1 - (np.count_nonzero(acc_array) / len(acc_array)) 

print(acc)

0.531578947368421


In [188]:
skill_team = dict(zip(teams, skill_list[:,0]))

# saving ranking 20/21
skill_team21_22 = skill_team
skill_team21_22

{'Genoa': 0.5762714726236828,
 'Chievo': -0.21013481043158128,
 'Bologna': 0.5425851113607536,
 'Atalanta': 1.3752663125476134,
 'Inter': 1.4471159064633174,
 'Empoli': 0.4953572554300179,
 'Udinese': 0.5021497563234768,
 'Lazio': 1.04922074598706,
 'Sampdoria': 0.8748870562840939,
 'Torino': 1.1772219968880175,
 'Spal': 0.6203526296042677,
 'Cagliari': 0.6581500470231323,
 'Milan': 1.2215409083212287,
 'Roma': 1.1085779537091927,
 'Sassuolo': 0.48960809051920073,
 'Juventus': 1.7529592307626687,
 'Fiorentina': 0.4365847432468136,
 'Frosinone': 0.12879295404838337,
 'Napoli': 1.5987088810579317,
 'Parma': 0.5592286592885929}

In [189]:
# sort dictionary by values in reverse order
ranking21_22 = dict(sorted(skill_team21_22.items(), key=lambda item: item[1], reverse=True))

ranking21_22 = pd.DataFrame.from_dict(ranking21_22, orient='index')
ranking21_22

0
Juventus    1.752959
Napoli      1.598709
Inter       1.447116
Atalanta    1.375266
Milan       1.221541
Torino      1.177222
Roma        1.108578
Lazio       1.049221
Sampdoria   0.874887
Cagliari    0.658150
Spal        0.620353
Genoa       0.576271
Parma       0.559229
Bologna     0.542585
Udinese     0.502150
Empoli      0.495357
Sassuolo    0.489608
Fiorentina  0.436585
Frosinone   0.128793
Chievo     -0.210135